In [1]:
library(tidyverse)
library(mediation)
library(brms)
library(dplyr)
library(tidyr)
library(lme4)
library(ggplot2)

── Attaching core tidyverse packages ─────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   4.0.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.1.0     
── Conflicts ───────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the ]8;;http://conflicted.r-lib.org/conflicted package]8;; to force all conflicts to become errors


Warning messages:
1: package ‘tidyverse’ was built under R version 4.3.3 
2: package ‘lubridate’ was built under R version 4.3.2 


Loading required package: MASS

Attaching package: ‘MASS’

The following object is masked from ‘package:dplyr’:

    select

Loading required package: Matrix

Attaching package: ‘Matrix’

The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack

Loading required package: mvtnorm
Loading required package: sandwich
mediation: Causal Mediation Analysis
Version: 4.5.1



Warning messages:
1: package ‘mvtnorm’ was built under R version 4.3.3 
2: package ‘sandwich’ was built under R version 4.3.3 


Loading required package: Rcpp
Loading 'brms' package (version 2.22.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').

Attaching package: ‘brms’

The following object is masked from ‘package:stats’:

    ar



Warning message:
package ‘brms’ was built under R version 4.3.3 



Attaching package: ‘lme4’

The following object is masked from ‘package:brms’:

    ngrps



# Import Dataset

## Import

In [2]:
df_long <- read.csv("df_RCT.csv")

# Use baseline visit rows
df <- df_long %>%
  filter(time_point == 1)

## Long Format

In [6]:
library(tidyr)
library(dplyr)

df_long_mip <- df %>%
  dplyr::select(record_id, mip_pre_max, mip_post_max) %>%
  pivot_longer(
    cols = c(mip_pre_max, mip_post_max),
    names_to = "time",
    values_to = "mip_max"
  ) %>%
  mutate(
    time = if_else(time == "mip_pre_max", 0, 1)  # 0 = pre, 1 = post
  )


# Outcome Measures

In [3]:
outcomes <- c("fss_sum", "woods_sum", "dsq_sum", "dsq_freq_sum", "dsq_severity_sum", "psqi_sum", "psqi_disturbances")

resp_vars <- c(
  "mip_pre_max", "smip_pre_max", "fit_pre_max", "id_pre_max", "slopesmip_pre_max",
  "sindex_pre_max", "pif_pre_max", "volume_pre_max",
  "mip_post_max", "smip_post_max", "fit_post_max", "id_post_max", "slopesmip_post_max",
  "pif_post_max", "sindex_post_max", "volume_post_max",
  "cpet_vo2peak_absolute", "cpet_ve", "cpet_vt_peak", "cpet_o2pulse",
  "fmd_percent"
)

covariates <- c("data_age", "data_centimeters", "data_kilograms", "subject_female")


# Responder vs Nonresponder

In [4]:
library(dplyr)

# Set your threshold  
change_threshold <- 10 

# Create responder variable
df_resp <- df %>%
  filter(!is.na(mip_max_fatigue)) %>%
  mutate(
    mip_change = mip_max_fatigue,
    responder = if_else(abs(mip_change) >= change_threshold,
                        "responder", "nonresponder")
  )

# Print counts
cat("\n--- RESPONDER VS NONRESPONDER ---\n")
print(table(df_resp$responder))

# Save table
responder_table <- as.data.frame(table(df_resp$responder))
write.csv(responder_table, "mip_responder_counts.csv", row.names = FALSE)

cat("\nSaved file: mip_responder_counts.csv\n")



--- RESPONDER VS NONRESPONDER ---

nonresponder    responder 
           9           13 

Saved file: mip_responder_counts.csv


# LMM

In [7]:
library(lme4)

model <- lmer(mip_max ~ time + (time | record_id), data = df_long_mip)
summary(model)


: Error:
! number of observations (=44) <= number of random effects (=44) for term (time | record_id); the random-effects parameters and the residual variance (or scale parameter) are probably unidentifiable

# Dispersion Analysis

# Reliable Change Index (RCI)

# End